# DESIGUALDADE DE RENDA E EDUCAÇÃO NAS REGIÕES METROPOLITANAS DO BRASIL: uma análise de dados em painel entre 2013 e 2023

# 1. Importando e visualizando os dados

## 1.1 Instalação e Carregamento de Pacotes

In [ ]:
# Instalação e Carregamento de Pacotes ---

# Serão utilizadas as seguintes bibliotecas:
# - rvest: Para ler e extrair links da página HTML do portal do IBGE.
# - httr: Para realizar o download dos arquivos de forma eficiente.
# - dplyr: Para manipulação de dados.
# - readr: Para a leitura dos dados após a extração.

if (!requireNamespace("rvest", quietly = TRUE)) {
  install.packages("rvest")
}
if (!requireNamespace("httr", quietly = TRUE)) {
  install.packages("httr")
}
if (!requireNamespace("dplyr", quietly = TRUE)) {
  install.packages("dplyr")
}
if (!requireNamespace("readr", quietly = TRUE)) {
  install.packages("readr")
}
if (!requireNamespace("bigrquery", quietly = TRUE)) {
  install.packages("bigrquery")
}

library(rvest)
library(httr)
library(dplyr)
library(readr)

In [ ]:
# Instalando o pacote PNADcIBGE
install.packages("PNADcIBGE")
install.packages("googledrive")

In [ ]:
# Carregando as bibliotecas que serão utilizadas no script.
library(PNADcIBGE)
library(dplyr)
library(googledrive)
library(readr)
library(bigrquery)
library(plm)

## 1.2 Fazendo o Web Scraping

In [ ]:
#-----------------------------------------------------------------------------
# PROJETO DE PESQUISA - IPEA
# SCRIPT DE COLETA DE DADOS COM UPLOAD PARA O GOOGLE DRIVE
#-----------------------------------------------------------------------------
# Este script utiliza o pacote 'PNADcIBGE' para baixar os microdados e o
# pacote 'googledrive' para fazer o upload automático para o Google Drive,
# resolvendo o problema de espaço local.
#
# Processo:
# 1. Autentica com sua conta Google.
# 2. Baixa os arquivos de um trimestre para uma pasta temporária.
# 3. Faz o upload dos arquivos para uma pasta específica no seu Google Drive.
# 4. Limpa os arquivos temporários locais para liberar espaço.
# 5. Repete o processo para todos os trimestres.
#-----------------------------------------------------------------------------

# Carregando as bibliotecas necessárias.
library(PNADcIBGE)
library(googledrive)
library(dplyr)

# --- PASSO 1: Autenticação com o Google Drive ---
#
# Este comando abrirá uma janela no seu navegador para você autorizar o R.
# Use o e-mail que você forneceu: mauricio.meira@ufv.br
# A autenticação fica salva para usos futuros.
drive_auth(email = "mauricio.meira@ufv.br")


# --- PASSO 2: Definição de Parâmetros e Pasta no Google Drive ---

anos_de_interesse <- 2013:2023
nome_pasta_drive <- "Dados_PNADc_IBGE_2013-2023"

# Verifica se a pasta de destino já existe no seu Google Drive.
# A função 'drive_find' retorna um 'dribble', um tipo de dataframe do pacote.
pasta_destino_info <- drive_find(n_max = 1, q = paste0("name = '", nome_pasta_drive, "' and mimeType = 'application/vnd.google-apps.folder' and trashed = false"))

# Se a pasta não existe (o dribble tem 0 linhas), cria a pasta.
if (nrow(pasta_destino_info) == 0) {
  cat(paste0("Criando a pasta '", nome_pasta_drive, "' no seu Google Drive...\n"))
  pasta_destino_info <- drive_mkdir(nome_pasta_drive)
} else {
  cat(paste0("Pasta '", nome_pasta_drive, "' já encontrada no Google Drive.\n"))
}

# Extrai o ID da pasta para usar nos uploads.
id_pasta_drive <- pasta_destino_info$id


# --- PASSO 3: Download Temporário e Upload para o Google Drive ---

for (ano in anos_de_interesse) {
  for (trimestre in 1:4) {

    # Cria um diretório temporário para cada download.
    # Este diretório será apagado ao final de cada iteração do loop.
    temp_dir <- file.path(tempdir(), paste0("pnadc_", ano, "_", trimestre))
    if (!dir.exists(temp_dir)) dir.create(temp_dir)

    tryCatch({

      cat("--------------------------------------------------\n")
      cat(paste0("Processando: ", ano, " - Trimestre ", trimestre, "\n"))

      # Baixa os dados para a pasta temporária local.
      get_pnadc(year = ano, quarter = trimestre, savedir = temp_dir)

      # Lista todos os arquivos que foram baixados e descompactados.
      arquivos_para_upload <- list.files(temp_dir, full.names = TRUE, recursive = TRUE)

      if (length(arquivos_para_upload) > 0) {
        cat("Iniciando upload para o Google Drive...\n")
        # Itera sobre cada arquivo e faz o upload para a pasta de destino.
        for (arquivo in arquivos_para_upload) {
          drive_upload(media = arquivo,
                       path = as_id(id_pasta_drive), # Usa o ID da pasta de destino
                       name = basename(arquivo),     # Mantém o nome original do arquivo
                       overwrite = TRUE)             # Sobrescreve se já existir
        }
        cat("Upload concluído para ", ano, "T", trimestre, ".\n")
      } else {
        cat("Nenhum arquivo para fazer upload para ", ano, "T", trimestre, ".\n")
      }

    }, error = function(e) {
      cat(paste0("AVISO: Não foi possível processar ", ano, "T", trimestre, ". Motivo: ", e$message, "\n"))
    }, finally = {
      # Limpa a pasta temporária para liberar espaço no seu computador.
      # O 'unlink' remove a pasta e todo o seu conteúdo.
      cat("Limpando arquivos temporários locais...\n")
      unlink(temp_dir, recursive = TRUE)
    })

    # Pausa para não sobrecarregar o servidor do IBGE.
    Sys.sleep(1)
  }
}

cat("--------------------------------------------------\n")
cat("PROCESSO DE COLETA DE DADOS E UPLOAD CONCLUÍDO.\n")
cat("Todos os arquivos foram salvos na pasta '", nome_pasta_drive, "' no seu Google Drive.\n")


## 1.3 Carregando os dados do google drive

In [6]:
# Carregando bibliotecas
suppressPackageStartupMessages({
  library(PNADcIBGE)
  library(googledrive)
  library(dplyr)
  library(readr)
})

# Suprimindo mensagens do googledrive
options(googledrive_quiet = TRUE)

# Parâmetros
anos_de_interesse <- 2013:2023
trimestres <- 1:4
nome_pasta_drive <- "Pnad_Contínua_2013-2023"
lista_de_dataframes <- list()

colunas_necessarias <- c("Ano", "Trimestre", "UF", "RM_RIDE", "Capital", "V2007",
                         "V2009", "V2010", "V3001", "VD3005", "VD4001", "VD4016",
                         "VD4017", "V2001", "posest", "posest_sxi")

# Buscando a pasta no Drive
pasta_query <- paste0("name='", nome_pasta_drive, "' and mimeType='application/vnd.google-apps.folder' and trashed=false")
pasta_drive <- drive_find(q = pasta_query, n_max = 1)

if (nrow(pasta_drive) == 0) {
  stop("❌ ERRO: Pasta '", nome_pasta_drive, "' não encontrada no Google Drive!")
}

# Buscando arquivo input SAS
input_query <- paste0("name='input_PNADC_trimestral.sas' and '", pasta_drive$id, "' in parents and trashed=false")
input_sas <- drive_find(q = input_query, n_max = 1)

if (nrow(input_sas) == 0) {
  stop("❌ ERRO: Arquivo 'input_PNADC_trimestral.sas' não encontrado!")
}

# Baixando arquivo input SAS
temp_input_sas <- tempfile(fileext = ".sas")
suppressMessages(drive_download(input_sas, path = temp_input_sas, overwrite = TRUE))

# Função para processar arquivos
processar_arquivo_pnad <- function(ano, trimestre, pasta_id, arquivo_input) {

  nome_arquivo <- paste0("PNADC_0", trimestre, ano, ".txt")
  arquivo_query <- paste0("name='", nome_arquivo, "' and '", pasta_id, "' in parents and trashed=false")
  arquivo_encontrado <- drive_find(q = arquivo_query, n_max = 1)

  if (nrow(arquivo_encontrado) == 0) {
    return(NULL)
  }

  temp_file <- tempfile(fileext = ".txt")

  tryCatch({
    # Download
    suppressMessages(drive_download(arquivo_encontrado, path = temp_file, overwrite = TRUE))

    # Lendo dados (SEM labels = TRUE para evitar erro)
    dados <- read_pnadc(microdata = temp_file, input_txt = arquivo_input)

    # Filtrando colunas
    dados_filtrados <- dados %>%
      select(any_of(colunas_necessarias))

    # Limpando arquivo temporário
    if(file.exists(temp_file)) file.remove(temp_file)

    return(dados_filtrados)

  }, error = function(e) {
    if(file.exists(temp_file)) file.remove(temp_file)
    return(NULL)
  })
}

# Processando todos os arquivos
cat("🔄 Carregando dados da PNAD... Aguarde.\n")

contador_sucesso <- 0
contador_total <- length(anos_de_interesse) * length(trimestres)

# Criando barra de progresso
cat("Progresso: [")

for (ano in anos_de_interesse) {
  for (trimestre in trimestres) {

    dados_trimestre <- processar_arquivo_pnad(ano, trimestre, pasta_drive$id, temp_input_sas)

    if (!is.null(dados_trimestre)) {
      lista_de_dataframes[[paste0(ano, "T", trimestre)]] <- dados_trimestre
      contador_sucesso <- contador_sucesso + 1
      cat("■")  # Progresso visual
    } else {
      cat("□")  # Arquivo não encontrado/processado
    }

    Sys.sleep(0.1)  # Pausa mínima
  }
}

cat("]\n")

# Limpando arquivo input temporário
if(file.exists(temp_input_sas)) file.remove(temp_input_sas)

# Verificando se obteve dados
if (length(lista_de_dataframes) == 0) {
  stop("❌ ERRO: Nenhum arquivo foi processado com sucesso!")
}

# Combinando e processando dados
suppressMessages({
  pnad_completa <- bind_rows(lista_de_dataframes)

  result <- pnad_completa %>%
    rename(
      ano = Ano, trimestre = Trimestre, unidade_federacao = UF,
      regiao_metropolitana = RM_RIDE, municipio_capital = Capital,
      sabe_ler_escrever = V3001, sexo = V2007, idade = V2009,
      cor_raca = V2010, anos_estudo = VD3005, condicao_ocupacao = VD4001,
      rendimento_habitual_principal = VD4016, renda_domiciliar_total = VD4017,
      numero_pessoas_domicilio = V2001
    ) %>%
    filter(
      !is.na(regiao_metropolitana), !is.na(rendimento_habitual_principal),
      !is.na(renda_domiciliar_total), !is.na(numero_pessoas_domicilio),
      !is.na(anos_estudo), !is.na(condicao_ocupacao), !is.na(sabe_ler_escrever),
      !is.na(idade), !is.na(sexo), !is.na(cor_raca)
    ) %>%
    mutate(
      rendimento_domiciliar_pc = renda_domiciliar_total / numero_pessoas_domicilio
    ) %>%
    group_by(ano, trimestre, regiao_metropolitana) %>%
    mutate(
      total_mais_15 = sum(idade >= 15, na.rm = TRUE),
      total_analfabetos = sum(sabe_ler_escrever == "Não" & idade >= 15, na.rm = TRUE),
      taxa_analfabetismo = ifelse(total_mais_15 > 0, total_analfabetos / total_mais_15, 0)
    ) %>%
    ungroup() %>%
    select(
      ano, trimestre, unidade_federacao_nome = unidade_federacao,
      regiao_metropolitana_nome = regiao_metropolitana,
      municipio_capital_nome = municipio_capital,
      rendimento_habitual_principal, rendimento_domiciliar_pc,
      numero_pessoas_domicilio, anos_estudo,
      sabe_ler_escrever_nome = sabe_ler_escrever, idade, sexo_nome = sexo,
      cor_raca_nome = cor_raca, condicao_ocupacao_nome = condicao_ocupacao,
      posest, posest_sxi, taxa_analfabetismo
    ) %>%
    arrange(regiao_metropolitana_nome, ano, trimestre)
})

# Visualizando o dataframe
result

🔄 Carregando dados da PNAD... Aguarde.
Progresso: [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■]


[1] TRUE

ano,trimestre,unidade_federacao_nome,regiao_metropolitana_nome,municipio_capital_nome,rendimento_habitual_principal,rendimento_domiciliar_pc,numero_pessoas_domicilio,anos_estudo,sabe_ler_escrever_nome,idade,sexo_nome,cor_raca_nome,condicao_ocupacao_nome,posest,posest_sxi,taxa_analfabetismo
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
2013,1,13,13,13,2200,733.33333,3,12,1,51,1,1,1,131,111,0
2013,1,13,13,13,840,280.00000,3,12,1,34,2,1,1,131,207,0
2013,1,13,13,13,1000,200.00000,5,16,1,32,2,4,1,131,207,0
2013,1,13,13,13,1400,280.00000,5,12,1,37,1,4,1,131,108,0
2013,1,13,13,13,500,100.00000,5,09,1,25,2,4,1,131,206,0
2013,1,13,13,13,700,233.33333,3,09,1,64,1,4,1,131,113,0
2013,1,13,13,13,678,226.00000,3,00,1,52,2,1,1,131,211,0
2013,1,13,13,13,500,55.55556,9,03,1,63,1,4,1,131,113,0
2013,1,13,13,13,200,22.22222,9,04,1,20,1,4,1,131,105,0


### 1.4 Tratando os valores ausentes e números decimais da coluna rendimento_domiciliar_pc

In [ ]:
# Excluir linhas que contêm NA na tabela 'result'
result_clean <- na.omit(result)

# Arredondar os valores da coluna 'rendimento_domiciliar_pc' para duas casas decimais
result_clean$rendimento_domiciliar_pc <- round(result_clean$rendimento_domiciliar_pc, 2)

# Verificar os primeiros registros da tabela limpa
head(result_clean, 8)

ano,trimestre,unidade_federacao_nome,regiao_metropolitana_nome,municipio_capital_nome,rendimento_habitual_principal,rendimento_domiciliar_pc,numero_pessoas_domicilio,anos_estudo,sabe_ler_escrever_nome,idade,sexo_nome,cor_raca_nome,condicao_ocupacao_nome,posest,posest_sxi,taxa_analfabetismo
<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
2013,1,22,Região Administrativa Integrada de Desenvolvimento da Grande Teresina (PI),Município de Teresina (PI),678,226.0,3,12,Sim,56,Mulher,Parda,Pessoas na força de trabalho,221,212,0.08405439
2013,1,22,Região Administrativa Integrada de Desenvolvimento da Grande Teresina (PI),Município de Teresina (PI),5000,1000.0,5,12,Sim,43,Homem,Parda,Pessoas na força de trabalho,221,109,0.08405439
2013,1,22,Região Administrativa Integrada de Desenvolvimento da Grande Teresina (PI),Município de Teresina (PI),778,389.0,2,12,Sim,39,Mulher,Preta,Pessoas na força de trabalho,221,208,0.08405439
2013,1,22,Região Administrativa Integrada de Desenvolvimento da Grande Teresina (PI),Município de Teresina (PI),100,25.0,4,8,Sim,15,Mulher,Parda,Pessoas na força de trabalho,221,204,0.08405439
2013,1,22,Região Administrativa Integrada de Desenvolvimento da Grande Teresina (PI),Município de Teresina (PI),400,400.0,1,15,Sim,21,Mulher,Parda,Pessoas na força de trabalho,221,205,0.08405439
2013,1,22,Região Administrativa Integrada de Desenvolvimento da Grande Teresina (PI),Município de Teresina (PI),1244,0.0,5,16,Sim,49,Mulher,Parda,Pessoas na força de trabalho,221,210,0.08405439
2013,1,22,Região Administrativa Integrada de Desenvolvimento da Grande Teresina (PI),Município de Teresina (PI),678,113.0,6,15,Sim,27,Homem,Parda,Pessoas na força de trabalho,221,106,0.08405439
2013,1,22,Região Administrativa Integrada de Desenvolvimento da Grande Teresina (PI),Município de Teresina (PI),678,135.6,5,4,Sim,45,Homem,Preta,Pessoas na força de trabalho,221,110,0.08405439


# 2. Analisando os dados

## 2.1 Calculando o Índice de Gini para cada Região Metropolitana e Ano

In [ ]:
# Suprimir mensagens e avisos durante o carregamento dos pacotes
suppressMessages(library(dplyr))
suppressMessages(library(ineq))

# Calcular o Índice de Gini para cada Região Metropolitana e Ano
gini_data <- result_clean %>%
  group_by(ano, regiao_metropolitana_nome) %>%
  summarise(indice_gini = suppressWarnings(Gini(rendimento_domiciliar_pc, na.rm = TRUE)), .groups = "drop")

# Visualizar os primeiros resultados
head(gini_data, 10) # visualizando as 10 primeiras linhas

ano,regiao_metropolitana_nome,indice_gini
<int>,<chr>,<dbl>
2013,Região Administrativa Integrada de Desenvolvimento da Grande Teresina (PI),0.5751285
2013,Região Metropolitana de Aracaju (SE),0.5825165
2013,Região Metropolitana de Belo Horizonte (MG),0.5971018
2013,Região Metropolitana de Belém (PA),0.5988353
2013,Região Metropolitana de Curitiba (PR),0.5479370
2013,Região Metropolitana de Florianópolis (SC),0.5469473
2013,Região Metropolitana de Fortaleza (CE),0.5459002
2013,Região Metropolitana de Goiânia (GO),0.5431216
2013,Região Metropolitana de Grande São Luís (MA),0.4966935


## 2.2 Estatística descritiva das variáveis analisadas

In [ ]:
# Instale os pacotes necessários (se ainda não instalados)
if (!requireNamespace("knitr", quietly = TRUE)) install.packages("knitr")
if (!requireNamespace("kableExtra", quietly = TRUE)) install.packages("kableExtra")

# Carregar as bibliotecas
library(knitr)
library(kableExtra)

# Adicionar os nomes originais das variáveis e nomes resumidos
tabela_2 <- data.frame(
  Nomes_Originais = c("indice_gini", "anos_estudo", "taxa_analfabetismo", "rendimento_habitual_principal"),
  Variáveis = c("GINI", "ESTUDO", "ANALFABETISMO", "RENDA"),
  Média = c(mean(gini_data$indice_gini, na.rm = TRUE),
            mean(result_clean$anos_estudo, na.rm = TRUE),
            mean(result_clean$taxa_analfabetismo, na.rm = TRUE),
            mean(result_clean$rendimento_habitual_principal, na.rm = TRUE)),
  Desvio_Padrão = c(sd(gini_data$indice_gini, na.rm = TRUE),
                    sd(result_clean$anos_estudo, na.rm = TRUE),
                    sd(result_clean$taxa_analfabetismo, na.rm = TRUE),
                    sd(result_clean$rendimento_habitual_principal, na.rm = TRUE)),
  Valor_Mínimo = c(min(gini_data$indice_gini, na.rm = TRUE),
                   min(result_clean$anos_estudo, na.rm = TRUE),
                   min(result_clean$taxa_analfabetismo, na.rm = TRUE),
                   min(result_clean$rendimento_habitual_principal, na.rm = TRUE)),
  Valor_Máximo = c(max(gini_data$indice_gini, na.rm = TRUE),
                   max(result_clean$anos_estudo, na.rm = TRUE),
                   max(result_clean$taxa_analfabetismo, na.rm = TRUE),
                   max(result_clean$rendimento_habitual_principal, na.rm = TRUE))
)

# Ajustar o formato numérico (transformar os valores diretamente para strings formatadas)
tabela_2$Média <- round(tabela_2$Média, 4)
tabela_2$Desvio_Padrão <- round(tabela_2$Desvio_Padrão, 4)
tabela_2$Valor_Mínimo <- round(tabela_2$Valor_Mínimo, 2)
tabela_2$Valor_Máximo <- format(round(tabela_2$Valor_Máximo, 2),
                                big.mark = ".",
                                decimal.mark = ",",
                                scientific = FALSE)

# Exibir a Tabela 2 no formato estilizado
tabela_html <- kable(
  tabela_2,
  format = "html",
  align = "c",
  col.names = c("Nomes Originais", "Variáveis", "Média", "Desvio Padrão", "Valor Mínimo", "Valor Máximo"),
  caption = "Tabela 2 - Estatística descritiva das variáveis analisadas"
) %>%
  kable_styling(full_width = FALSE, bootstrap_options = c("striped", "hover", "condensed", "responsive"))

# Renderizar a tabela HTML no Jupyter Notebook
IRdisplay::display_html(as.character(tabela_html))

Nomes Originais,Variáveis,Média,Desvio Padrão,Valor Mínimo,Valor Máximo
indice_gini,GINI,0.5837,0.0324,0.48,"0,66"
anos_estudo,ESTUDO,11.7926,3.8003,0.00,"16,00"
taxa_analfabetismo,ANALFABETISMO,0.0217,0.0185,0.00,"0,09"
rendimento_habitual_principal,RENDA,2636.7909,4061.3754,2.00,"300.000,00"


## 2.3 Painel de Dados Agregados: Índice de Gini, Anos de Estudo, Taxa de Analfabetismo e Rendimento Habitual por Região Metropolitana e Ano

In [ ]:
# Carregar a biblioteca dplyr, se ainda não estiver carregada
library(dplyr)

# Criar variáveis agregadas (médias/proporções), incluindo as variáveis de controle, e adicionar o Índice de Gini
painel_dados <- result_clean %>%
  group_by(ano, regiao_metropolitana_nome) %>%
  summarise(
    media_anos_estudo = mean(anos_estudo, na.rm = TRUE),
    proporcao_analfabetismo = mean(taxa_analfabetismo, na.rm = TRUE),
    media_rendimento_habitual = mean(rendimento_habitual_principal, na.rm = TRUE),
    media_idade = mean(idade, na.rm = TRUE), # Média da idade
    proporcao_homens = mean(sexo_nome == "Homem", na.rm = TRUE), # Proporção de homens
    proporcao_mulheres = mean(sexo_nome == "Mulher", na.rm = TRUE), # Proporção de mulheres
    proporcao_raca_branca = mean(cor_raca_nome == "Branca", na.rm = TRUE), # Proporção de pessoas brancas
    proporcao_raca_preta = mean(cor_raca_nome == "Preta", na.rm = TRUE), # Proporção de pessoas pretas
    proporcao_ocupacao = mean(condicao_ocupacao_nome == "Pessoas na força de trabalho", na.rm = TRUE), # Proporção de ocupação
    .groups = "drop"
  ) %>%
  # Unir com os dados do Índice de Gini
  left_join(
    gini_data %>% select(ano, regiao_metropolitana_nome, indice_gini),
    by = c("ano", "regiao_metropolitana_nome")
  ) %>%
  # Reordenar as colunas para que indice_gini fique após regiao_metropolitana_nome
  select(ano, regiao_metropolitana_nome, indice_gini, everything())

# Visualizar os primeiros resultados do painel ajustado
head(painel_dados, 10) # Exibe as primeiras 10 linhas

ano,regiao_metropolitana_nome,indice_gini,media_anos_estudo,proporcao_analfabetismo,media_rendimento_habitual,media_idade,proporcao_homens,proporcao_mulheres,proporcao_raca_branca,proporcao_raca_preta,proporcao_ocupacao
<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2013,Região Administrativa Integrada de Desenvolvimento da Grande Teresina (PI),0.5751285,10.40049,0.088382810,1433.699,38.39048,0.5474430,0.4525570,0.1776032,0.09041898,1
2013,Região Metropolitana de Aracaju (SE),0.5825165,11.14395,0.029491562,1889.039,37.56070,0.5369438,0.4630562,0.2848802,0.07630360,1
2013,Região Metropolitana de Belo Horizonte (MG),0.5971018,11.54438,0.012496778,2201.754,38.93911,0.5166750,0.4833250,0.4254189,0.10262154,1
2013,Região Metropolitana de Belém (PA),0.5988353,10.48676,0.019352519,1393.343,38.17251,0.5630460,0.4369540,0.2271483,0.07784587,1
2013,Região Metropolitana de Curitiba (PR),0.5479370,11.90481,0.011398003,2345.740,37.86694,0.5292238,0.4707762,0.8015411,0.03617741,1
2013,Região Metropolitana de Florianópolis (SC),0.5469473,11.98663,0.007832669,2368.221,37.85555,0.5341061,0.4658939,0.8649131,0.05840392,1
2013,Região Metropolitana de Fortaleza (CE),0.5459002,10.52500,0.049131694,1448.101,37.90016,0.5406352,0.4593648,0.3070033,0.05032573,1
2013,Região Metropolitana de Goiânia (GO),0.5431216,11.21903,0.017232735,2022.800,37.76633,0.5506266,0.4493734,0.4390596,0.06731729,1
2013,Região Metropolitana de Grande São Luís (MA),0.4966935,11.09660,0.023079870,1163.635,37.93111,0.5542971,0.4457029,0.1934710,0.12298468,1


# 3. Estimando o modelo econométrico de Efeitos Fixos

In [ ]:
# Carregar o pacote plm (se ainda não estiver carregado)
library(plm)

# Log-transformar variáveis principais para o modelo
painel_dados <- painel_dados %>%
  mutate(
    ln_indice_gini = log(indice_gini),
    ln_media_anos_estudo = log(media_anos_estudo),
    ln_proporcao_analfabetismo = log(proporcao_analfabetismo),
    ln_media_rendimento_habitual = log(media_rendimento_habitual)
  )

# Estimar o modelo de efeitos fixos (ou altere para "random" se necessário)
modelo <- plm(
  ln_indice_gini ~ ln_media_anos_estudo + ln_proporcao_analfabetismo + ln_media_rendimento_habitual +
    media_idade + proporcao_homens + proporcao_mulheres +
    proporcao_raca_branca + proporcao_raca_preta + proporcao_ocupacao,
  data = painel_dados,
  index = c("regiao_metropolitana_nome", "ano"),
  model = "within" # Para efeitos fixos; altere para "random" para efeitos aleatórios
)
# Instalar o pacote stargazer, se ainda não estiver instalado
if (!requireNamespace("stargazer", quietly = TRUE)) install.packages("stargazer")

# Carregar o pacote stargazer
library(stargazer)

# Exibir o modelo de forma mais formatada
stargazer(
  modelo,
  type = "text", # Use "html" para saída mais estilizada em navegadores ou relatórios
  title = "Resultados do Modelo de Efeitos Fixos",
  align = TRUE,
  dep.var.labels = "Log do Índice de Gini",
  covariate.labels = c(
    "Log dos Anos de Estudo (Média)",
    "Log da Proporção de Analfabetismo",
    "Log da Renda Média Habitual",
    "Idade Média",
    "Proporção de Homens",
    "Proporção de Mulheres",
    "Proporção de Raça Branca",
    "Proporção de Raça Preta",
    "Proporção na Força de Trabalho"
  ),
  omit.stat = c("f", "ser"), # Excluir estatísticas não relevantes
  digits = 3 # Controle do número de casas decimais
)


Resultados do Modelo de Efeitos Fixos
                                      Dependent variable:    
                                  ---------------------------
                                     Log do Índice de Gini   
-------------------------------------------------------------
Log dos Anos de Estudo (Média)              -0.020           
                                            (0.165)          
                                                             
Log da Proporção de Analfabetismo            0.007           
                                            (0.012)          
                                                             
Log da Renda Média Habitual                0.093***          
                                            (0.028)          
                                                             
Idade Média                                  0.001           
                                            (0.005)          
                               

# 4.Diagnóstico do Modelo

In [ ]:
# Carregar pacotes necessários
library(lmtest)

# Teste de heterocedasticidade (Breusch-Pagan)
hetero_test <- bptest(modelo)

# Teste de autocorrelação serial (Breusch-Godfrey/Wooldridge)
autocor_test <- pbgtest(modelo)

# Teste de Hausman (corrigido)
hausman_test <- phtest(
  ln_indice_gini ~ ln_media_anos_estudo + ln_proporcao_analfabetismo + ln_media_rendimento_habitual +
    media_idade + proporcao_homens + proporcao_mulheres +
    proporcao_raca_branca + proporcao_raca_preta + proporcao_ocupacao,
  data = painel_dados,
  index = c("regiao_metropolitana_nome", "ano"),
  method = "chisq" # Método correto
)

# Criar uma tabela com os resultados dos testes
resultados_teste <- data.frame(
  Teste = c("Heterocedasticidade (Breusch-Pagan)",
            "Autocorrelação Serial (Breusch-Godfrey/Wooldridge)",
            "Hausman"),
  Estatística = c(hetero_test$statistic, autocor_test$statistic, hausman_test$statistic),
  `Graus de Liberdade` = c(hetero_test$parameter, autocor_test$parameter, hausman_test$parameter),
  `P-Valor` = c(hetero_test$p.value, autocor_test$p.value, hausman_test$p.value),
  Interpretação = c(
    ifelse(hetero_test$p.value < 0.05, "Heterocedasticidade detectada", "Sem evidência de heterocedasticidade"),
    ifelse(autocor_test$p.value < 0.05, "Autocorrelação serial detectada", "Sem evidência de autocorrelação serial"),
    ifelse(hausman_test$p.value < 0.05, "Modelo de efeitos fixos preferido", "Modelo de efeitos aleatórios preferido")
  )
    )

resultados_teste %>%
  kable(
    format = "markdown",
    caption = "Resultados dos Testes de Diagnóstico",
    col.names = c("Teste", "Estatística", "GL", "P-Valor", "Interpretação"),
    align = "c"
  )




Table: Resultados dos Testes de Diagnóstico

|                       Teste                        | Estatística | GL |  P-Valor  |             Interpretação              |
|:--------------------------------------------------:|:-----------:|:--:|:---------:|:--------------------------------------:|
|        Heterocedasticidade (Breusch-Pagan)         |  34.739114  | 7  | 0.0000125 |     Heterocedasticidade detectada      |
| Autocorrelação Serial (Breusch-Godfrey/Wooldridge) |  51.119252  | 11 | 0.0000004 |    Autocorrelação serial detectada     |
|                      Hausman                       |  9.729663   | 7  | 0.2044178 | Modelo de efeitos aleatórios preferido |

# 5. Corrigindo a heterocedasticidade e autocorrelação serial detectados

In [ ]:
# Carregar o pacote necessário, se ainda não estiver carregado
if (!requireNamespace("plm", quietly = TRUE)) install.packages("plm")
if (!requireNamespace("lmtest", quietly = TRUE)) install.packages("lmtest")
if (!requireNamespace("stargazer", quietly = TRUE)) install.packages("stargazer")

# Reestimar o modelo utilizando estimador de efeitos fixos
library(plm)
modelo_robusto <- plm(
  ln_indice_gini ~ ln_media_anos_estudo + ln_proporcao_analfabetismo + ln_media_rendimento_habitual +
    media_idade + proporcao_homens + proporcao_mulheres +
    proporcao_raca_branca + proporcao_raca_preta + proporcao_ocupacao,
  data = painel_dados,
  index = c("regiao_metropolitana_nome", "ano"),
  model = "within" # Modelo de efeitos fixos
)

# Ajustar erros padrão robustos para Driscoll-Kraay
library(lmtest)
coefs_robustos <- coeftest(modelo_robusto, vcov = vcovSCC(modelo_robusto, type = "HC1", maxlag = 2))

# Apresentar os resultados de forma mais legível
library(stargazer)
stargazer(
  modelo_robusto,
  type = "text",
  se = list(sqrt(diag(vcovSCC(modelo_robusto, type = "HC1", maxlag = 2)))),
  title = "Resultados do Modelo Robusto (Driscoll-Kraay)",
  align = TRUE
)

# Mostrar os coeficientes ajustados com os erros padrão robustos
print(coefs_robustos)


Resultados do Modelo Robusto (Driscoll-Kraay)
                                 Dependent variable:    
                             ---------------------------
                                   ln_indice_gini       
--------------------------------------------------------
ln_media_anos_estudo                   -0.020           
                                       (0.157)          
                                                        
ln_proporcao_analfabetismo              0.007           
                                       (0.010)          
                                                        
ln_media_rendimento_habitual           0.093**          
                                       (0.046)          
                                                        
media_idade                             0.001           
                                       (0.008)          
                                                        
proporcao_homens                        0

# 6. Refazendo o diagnóstico do modelo

In [ ]:
# Modelo de efeitos fixos
modelo_fixo <- plm(
  ln_indice_gini ~ ln_media_anos_estudo + ln_proporcao_analfabetismo + ln_media_rendimento_habitual +
    media_idade + proporcao_homens + proporcao_mulheres +
    proporcao_raca_branca + proporcao_raca_preta + proporcao_ocupacao,
  data = painel_dados,
  index = c("regiao_metropolitana_nome", "ano"),
  model = "within" # Modelo de efeitos fixos
)

# Modelo de efeitos aleatórios
modelo_aleatorio <- plm(
  ln_indice_gini ~ ln_media_anos_estudo + ln_proporcao_analfabetismo + ln_media_rendimento_habitual +
    media_idade + proporcao_homens + proporcao_mulheres +
    proporcao_raca_branca + proporcao_raca_preta + proporcao_ocupacao,
  data = painel_dados,
  index = c("regiao_metropolitana_nome", "ano"),
  model = "random" # Modelo de efeitos aleatórios
)

# Realizar o teste de Hausman
hausman_teste <- phtest(modelo_fixo, modelo_aleatorio)

# Exibir os resultados do teste de Hausman
print(hausman_teste)


	Hausman Test

data:  ln_indice_gini ~ ln_media_anos_estudo + ln_proporcao_analfabetismo +  ...
chisq = 9.7297, df = 7, p-value = 0.2044
alternative hypothesis: one model is inconsistent



# 7. Estimando um modelo de efeitos aleatórios

In [ ]:
# Instalar e carregar o pacote stargazer, se necessário
if (!requireNamespace("stargazer", quietly = TRUE)) install.packages("stargazer")
library(stargazer)

# Exibir o modelo de forma organizada
stargazer(
  modelo_aleatorio,
  type = "text",
  title = "Resultados do Modelo de Efeitos Aleatórios",
  dep.var.labels = "Log do Índice de Gini",
  covariate.labels = c(
    "Log dos Anos de Estudo (Média)",
    "Log da Proporção de Analfabetismo",
    "Log da Renda Média Habitual",
    "Idade Média",
    "Proporção de Homens",
    "Proporção de Raça Branca",
    "Proporção de Raça Preta"
  ),
  digits = 3,
  out.header = FALSE,
  align = TRUE
)


Resultados do Modelo de Efeitos Aleatórios
                                      Dependent variable:    
                                  ---------------------------
                                     Log do Índice de Gini   
-------------------------------------------------------------
Log dos Anos de Estudo (Média)              -0.075           
                                            (0.152)          
                                                             
Log da Proporção de Analfabetismo            0.015           
                                            (0.010)          
                                                             
Log da Renda Média Habitual                0.098***          
                                            (0.027)          
                                                             
Idade Média                                  0.003           
                                            (0.005)          
                          

# 8. Diagnóstico do Modelo de Efeitos Aleatórios

In [ ]:
# Instalar e carregar pacotes necessários
if (!requireNamespace("lmtest", quietly = TRUE)) install.packages("lmtest")
library(lmtest)

# Teste de heterocedasticidade para o modelo de efeitos aleatórios
teste_heterocedasticidade <- bptest(modelo_aleatorio)

# Teste de autocorrelação serial para o modelo de efeitos aleatórios
teste_autocorrelacao <- pbgtest(modelo_aleatorio)

# Criar tabela de resultados
resultados_teste_aleatorio <- data.frame(
  Teste = c(
    "Heterocedasticidade (Breusch-Pagan)",
    "Autocorrelação Serial (Breusch-Godfrey/Wooldridge)"
  ),
  Estatística = c(
    teste_heterocedasticidade$statistic,
    teste_autocorrelacao$statistic
  ),
  GL = c(
    teste_heterocedasticidade$parameter,
    teste_autocorrelacao$parameter
  ),
  `P-Valor` = c(
    teste_heterocedasticidade$p.value,
    teste_autocorrelacao$p.value
  ),
  Interpretação = c(
    ifelse(teste_heterocedasticidade$p.value < 0.05, "Heterocedasticidade detectada", "Sem heterocedasticidade"),
    ifelse(teste_autocorrelacao$p.value < 0.05, "Autocorrelação serial detectada", "Sem autocorrelação serial")
  )
)

# Visualizar a tabela de resultados
resultados_teste_aleatorio %>%
  knitr::kable(
    format = "markdown",
    caption = "Resultados dos Testes de Diagnóstico (Modelo de Efeitos Aleatórios)",
    col.names = c("Teste", "Estatística", "GL", "P-Valor", "Interpretação"),
    align = "c"
  )



Table: Resultados dos Testes de Diagnóstico (Modelo de Efeitos Aleatórios)

|      |                       Teste                        | Estatística | GL | P-Valor  |          Interpretação          |
|:-----|:--------------------------------------------------:|:-----------:|:--:|:--------:|:-------------------------------:|
|BP    |        Heterocedasticidade (Breusch-Pagan)         |  34.73911   | 7  | 1.25e-05 |  Heterocedasticidade detectada  |
|chisq | Autocorrelação Serial (Breusch-Godfrey/Wooldridge) |  48.96352   | 11 | 1.00e-06 | Autocorrelação serial detectada |

# 9.  Reestimando o modelo de efeitos aleatórios utilizando erros-padrão robustos pelo método de Driscoll-Kraay para corrigir os problemas

In [ ]:
# Carregar pacotes necessários
if (!requireNamespace("plm", quietly = TRUE)) install.packages("plm")
if (!requireNamespace("lmtest", quietly = TRUE)) install.packages("lmtest")
if (!requireNamespace("stargazer", quietly = TRUE)) install.packages("stargazer")
if (!requireNamespace("sandwich", quietly = TRUE)) install.packages("sandwich")

library(plm)        # Modelos de painel
library(lmtest)     # Testes e coeftest
library(sandwich)   # Matrizes robustas
library(stargazer)  # Apresentação dos resultados

# Estimar o modelo de efeitos aleatórios
modelo_aleatorio <- plm(
  ln_indice_gini ~ ln_media_anos_estudo + ln_proporcao_analfabetismo +
    ln_media_rendimento_habitual + media_idade + proporcao_homens +
    proporcao_raca_branca + proporcao_raca_preta,
  data = painel_dados,
  index = c("regiao_metropolitana_nome", "ano"),
  model = "random"
)

# Ajustar erros-padrão robustos pelo método Driscoll-Kraay
erro_padrao_driscoll <- vcovSCC(modelo_aleatorio, type = "HC1", cluster = "time", maxlag = 2)

# Formatar a tabela com Stargazer
stargazer(
  modelo_aleatorio,
  type = "text",
  se = list(sqrt(diag(erro_padrao_driscoll))), # Erros robustos
  title = "Resultados do Modelo de Efeitos Aleatórios com Erros-Padrão Robustos (Driscoll-Kraay)",
  align = TRUE,
  digits = 4,
  omit.stat = c("f", "ser"), # Remove estatísticas não relevantes
  header = FALSE
)

# Gerar resultados detalhados com coeficientes, valores t e p-valores
resultados_robustos <- coeftest(modelo_aleatorio, vcov = erro_padrao_driscoll)

# Exibir os resultados detalhados
print("Resultados do Modelo de Efeitos Aleatórios com Erros-Padrão Robustos (Driscoll-Kraay):")
print(resultados_robustos)


Resultados do Modelo de Efeitos Aleatórios com Erros-Padrão Robustos (Driscoll-Kraay)
                                 Dependent variable:    
                             ---------------------------
                                   ln_indice_gini       
--------------------------------------------------------
ln_media_anos_estudo                   -0.0747          
                                      (0.1489)          
                                                        
ln_proporcao_analfabetismo             0.0147           
                                      (0.0100)          
                                                        
ln_media_rendimento_habitual           0.0982*          
                                      (0.0578)          
                                                        
media_idade                            0.0030           
                                      (0.0083)          
                                                        
p